In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import cpca
import os
os.chdir("/Users/anorawu/BFI Dropbox/Wanru Wu/Cloudseeding/data/抗议数据")


In [84]:
df = pd.read_csv("/Users/anorawu/BFI Dropbox/Wanru Wu/Cloudseeding/data/抗议数据/RFA_protest3.csv",encoding='utf-8')
df = df[['adcode','location','size_level','year','month','day','citycode','省','市','区']]
df.to_csv("/Users/anorawu/BFI Dropbox/Wanru Wu/Cloudseeding/data/抗议数据/RFA_protest3_cropped.csv",index=False)

In [87]:
df = pd.read_stata("cloudseeding.dta")
df["district"] = df['prov'] + df['city'] + df['county']
df_adcode = cpca.transform(df["district"])
df['adcode'] = df_adcode['adcode']
df.to_csv("cloudseeding_adcode.csv")

In [12]:
df = pd.read_stata("Meteorological.dta")
df["district"] = df['prov'] + df['city'] + df['county']
df_adcode = cpca.transform(df["district"])
df['adcode'] = df_adcode['adcode']
df.to_csv("meteorological_adcode.csv")

In [2]:
df = pd.read_stata("eventstudy.dta")
df = df.sort_values(by=['citycode', 'date']).reset_index()
df["day"] = df.groupby("citycode").cumcount()
df['event'] = 0

# Identify events (first protest and subsequent protests >= 3 months apart)
for city, city_df in df.groupby('citycode'):
    protest_dates = city_df.loc[city_df['n_prt_weibo'] > 0, 'day'].sort_values().tolist()
    last_event = None
    
    for protest_date in protest_dates:
        if last_event is None or (protest_date - last_event) >= 45:
            df.loc[(df['citycode'] == int(city)) & (df['day'] == protest_date),'event'] = 1
            last_event = protest_date

index_list = df.loc[df['event']==1,'index'].tolist()
df['to_day']=None
for index in index_list:
    for i in range(-22,24):
        if not (index+i<0) or (index+i>len(df)):
            df.loc[df['index']==index+i,'to_day'] = i

df.to_csv('eventstudy.csv')